In [1]:
from pathlib import Path
import pandas as pd

from make_summary import summarize_group
from mil.utils import human_format

yaml_folder = Path("conf") / "selected_model" / "mnist_collage"

/Users/georg/Projects/mil/mil_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = [file.stem for file in yaml_folder.glob("*.yaml") if file.name[0] != "_"]
models.sort()
models

['abmil',
 'discrete_rel_pos_self_attention',
 'distance_aware_self_attention',
 'gnn_gat',
 'gnn_gcn',
 'induced_set_transformer',
 'just_pool',
 'mil_gnn',
 'mil_gnn_ds',
 'self_attention',
 'self_attention_axial_pe',
 'self_attention_fourier_pe',
 'set_transformer',
 'transmil']

In [3]:
mnist_collage_stats = {
    model: summarize_group(f"selected-mnist_collage-{model}", log_to_wandb=False)
    for model in models
}
mnist_collage_inverse_stats = {
    model: summarize_group(f"selected-mnist_collage_inverse-{model}", log_to_wandb=False)
    for model in models
}

2023-05-06 23:47:36.286 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-abmil
2023-05-06 23:47:38.722 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-discrete_rel_pos_self_attention
2023-05-06 23:47:41.067 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-distance_aware_self_attention
2023-05-06 23:47:43.482 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-gnn_gat
2023-05-06 23:47:45.550 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-gnn_gcn
2023-05-06 23:47:47.745 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-induced_set_transformer
2023-05-06 23:47:50.210 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-mnist_collage-just_pool
2023-05-06 23:47:52.350 | INFO     | make_summary:summarize_group:43 - Summarizing group selected-

In [4]:
df_mnist_collage = pd.DataFrame(mnist_collage_stats).T
df_mnist_collage_inverse = pd.DataFrame(mnist_collage_inverse_stats).T

# Merge
df_mnist_collage["dataset"] = "mnist_collage"
df_mnist_collage_inverse["dataset"] = "mnist_collage_inverse"
df = pd.concat([df_mnist_collage, df_mnist_collage_inverse])
df.index.name = "model"
df.reset_index(inplace=True)
df.set_index(["dataset", "model"], inplace=True)
df

mean(train/acc)  \
dataset               model                                              
mnist_collage         abmil                                   0.798667   
                      discrete_rel_pos_self_attention         0.921333   
                      distance_aware_self_attention           0.932000   
                      gnn_gat                                 0.810667   
                      gnn_gcn                                 0.864667   
                      induced_set_transformer                 0.812667   
                      just_pool                               0.846000   
                      mil_gnn                                 0.546000   
                      mil_gnn_ds                              0.686000   
                      self_attention                          0.882000   
                      self_attention_axial_pe                 0.862000   
                      self_attention_fourier_pe               0.878000   
                      set_transformer                         0.815333   
                      transmil                                0.957333   
mnist_collage_inverse abmil                                   0.804667   
                      discrete_rel_pos_self_attention         0.877333   
                      distance_aware_self_attention           0.910000   
                      gnn_gat                                 0.744667   
                      gnn_gcn                                 0.882667   
                      induced_set_transformer                 0.813333   
                      just_pool                               0.840000   
                      mil_gnn                                 0.542667   
                      mil_gnn_ds                              0.784000   
                      self_attention                          0.876000   
                      self_attention_axial_pe                 0.902000   
                      self_attention_fourier_pe               0.878667   
                      set_transformer                         0.818667   
                      transmil                                0.956667   

                                                       std(train/acc)  \
dataset               model                                             
mnist_collage         abmil                                  0.013864   
                      discrete_rel_pos_self_attention        0.013864   
                      distance_aware_self_attention          0.028829   
                      gnn_gat                                0.089362   
                      gnn_gcn                                0.030876   
                      induced_set_transformer                0.016898   
                      just_pool                              0.021266   
                      mil_gnn                                0.060892   
                      mil_gnn_ds                             0.143225   
                      self_attention                         0.041001   
                      self_attention_axial_pe                0.007674   
                      self_attention_fourier_pe              0.034529   
                      set_transformer                        0.051779   
                      transmil                               0.029477   
mnist_collage_inverse abmil                                  0.006055   
                      discrete_rel_pos_self_attention        0.031127   
                      distance_aware_self_attention          0.048362   
                      gnn_gat                                0.033383   
                      gnn_gcn                                0.033780   
                      induced_set_transformer                0.022730   
                      just_pool                              0.009129   
                      mil_gnn                                0.095406   
                      mil_gnn_ds                             0.022779   
         

In [5]:
NAMES = {
    "just_pool": "MIL with max pool",
    "abmil": "AB-MIL~\\cite{ilse2018attention}",
    "gnn_gat": "MIL with GNN (GAT~\\cite{velickovic2018graph})",
    "gnn_gcn": "MIL with GNN (GCN~\\cite{kipf2017semisupervised})",
    "induced_set_transformer": "MIL with iSet Transformer~\\cite{lee2019set}",
    "set_transformer": "MIL with Set Transformer~\\cite{lee2019set}",
    "mil_gnn": "MIL-GNN~\\cite{tu2019multiple}",
    "mil_gnn_ds": "MIL-GNN-DS~\\cite{tu2019multiple}",
    "self_attention": "MIL with SA~\\cite{vaswani2017attention}",
    "self_attention_axial_pe": "MIL with SA + axial PE~\\cite{ramachandran2019stand}",
    "self_attention_fourier_pe": "MIL with SA + Fourier PE~\\cite{yang2021learnable}",
    "discrete_rel_pos_self_attention": "MIL with disc.\\ rel.\\ SA~\\cite{wu2021rethinking}",
    "transmil": "Trans-MIL~\\cite{shao2021transmil}",
    "distance_aware_self_attention": "DAS-MIL (ours)",
}
abs_pos = ["self_attention_axial_pe", "self_attention_fourier_pe", "transmil"]
rel_pos = ["discrete_rel_pos_self_attention", "distance_aware_self_attention", "mil_gnn", "mil_gnn_ds", "gnn_gat", "gnn_gcn"]

   

print("\\begin{tabular}{l|c|r|rr|rr}")
print("\\toprule")
print(" & & & \\multicolumn{2}{c|}{\\smaller{MNIST-COLLAGE}} & \\multicolumn{2}{c}{\\smaller{MNIST-COLLAGE-INV}} \\\\")
print("Model & \\multicolumn{1}{c|}{Pos} & \\multicolumn{1}{c|}{Params} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c|}{Test} & \\multicolumn{1}{c}{Train} & \\multicolumn{1}{c}{Test} \\\\")
print("\\midrule")

rows = {name: row for name, row in df.iterrows()}
for name, desc in NAMES.items():
    pos = "abs" if name in abs_pos else "rel" if name in rel_pos else None
    row = f"{NAMES[name]}"
    pos = f" \\textsc{{{pos}}}" if pos in ("abs", "rel") else "\\xmark"
    row += f" & {pos}"
    row += f" & {human_format(df.loc['mnist_collage', name]['num_parameters'])}"

    metric = "balanced_acc"

    for dataset in ("mnist_collage", "mnist_collage_inverse"):
        df_ds = df.loc[dataset]
        for split in ("train", "test"):
            if df_ds[f"mean({split}/{metric})"].max() == df_ds[f"mean({split}/{metric})"].loc[name]:
                row += f" & \\textbf{{{df_ds[f'mean({split}/{metric})'].loc[name]:.03f} $\pm$ {df_ds[f'std({split}/{metric})'].loc[name]:.03f}}}"
            else:
                row += f" & {df_ds[f'mean({split}/{metric})'].loc[name]:.03f} $\pm$ {df_ds[f'std({split}/{metric})'].loc[name]:.03f}"
    print(row + " \\\\")
print("\\bottomrule")
print("\\end{tabular}")

\begin{tabular}{l|c|r|rr|rr}
\toprule
 & & & \multicolumn{2}{c|}{\smaller{MNIST-COLLAGE}} & \multicolumn{2}{c}{\smaller{MNIST-COLLAGE-INV}} \\
Model & \multicolumn{1}{c|}{Pos} & \multicolumn{1}{c|}{Params} & \multicolumn{1}{c}{Train} & \multicolumn{1}{c|}{Test} & \multicolumn{1}{c}{Train} & \multicolumn{1}{c}{Test} \\
\midrule
MIL with max pool & \xmark & 15.6K & 0.846 $\pm$ 0.021 & 0.828 $\pm$ 0.033 & 0.840 $\pm$ 0.009 & 0.788 $\pm$ 0.029 \\
AB-MIL~\cite{ilse2018attention} & \xmark & 16.1K & 0.799 $\pm$ 0.014 & 0.740 $\pm$ 0.010 & 0.805 $\pm$ 0.006 & 0.692 $\pm$ 0.015 \\
MIL with GNN (GAT~\cite{velickovic2018graph}) &  \textsc{rel} & 16.7K & 0.811 $\pm$ 0.089 & 0.758 $\pm$ 0.041 & 0.745 $\pm$ 0.033 & 0.716 $\pm$ 0.018 \\
MIL with GNN (GCN~\cite{kipf2017semisupervised}) &  \textsc{rel} & 16.3K & 0.865 $\pm$ 0.031 & 0.790 $\pm$ 0.058 & 0.883 $\pm$ 0.034 & 0.794 $\pm$ 0.036 \\
MIL with iSet Transformer~\cite{lee2019set} & \xmark & 22.1K & 0.813 $\pm$ 0.017 & 0.734 $\pm$ 0.040 & 0.813 $\p